# 효율적인 소수 찾기 알고리즘

Reference:
* [1]. [Why do we check up to the square root of a prime number to determine if it is prime?](https://stackoverflow.com/questions/5811151/why-do-we-check-up-to-the-square-root-of-a-prime-number-to-determine-if-it-is-pr)
* [2]. [(파이썬) 소수 판별하기](http://53perc.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%86%8C%EC%88%98-%ED%8C%90%EB%B3%84%ED%95%98%EA%B8%B0)

<요약>
* 짝수와 제곱근을 활용하여 약수를 찾는 숫자의 범위를 줄여 소수 감별기의 속도를 높였습니다.
* 첫 번째 자리수부터 소수 감별을 실시하고, 아닐 경우 해당 자리의 수의 숫자를 1 감소합니다.
* 낮은 자리가 아닌 높은 자리에서의 수를 우선적으로 감소시킴으로써 검색 범위를 대폭 축소할 수 있습니다. 

## 1. 입력된 자연수가 소수인지 판별하시오.

In [6]:
import sys, os
import time

In [1]:
def check_prime(nat_num):    
    if nat_num == 1: # 1은 소수가 아님.
        return False
    elif nat_num == 2: # 2는 가장 작은 소수임.
        return True
    
    if nat_num % 2 == 0: # 2를 제외한 모든 짝수는 2로 나눠지므로 소수가 아니다
        return False
    
    ### 효율적인 소수 찾기 아이디어
    # 만약 n이 소수가 아니라면, n = a*b로 표현된다 (여기서 a와 b는 1 또는 자기자신이 아님)
    # (우리는 a만 고려해서) a가 1 또는 자기자신이 아닌 것만 찾으면 된다 (소수가 아닌 조건)
    # 다시 말해, 일반적으로 n이하의 모든 숫자들을 "나머지0" 인지 체크하는 것이 아닌,
    # 가능한 a의 숫자들만 "나머지0" 인지 체크해보자 (체크 경우의 수를 훨씬 더 줄일 수 있다)
    # 여기서 중요한 점이 모든 경우의 a는 루트n 범위 내에 있을 수 밖에 없다. (n이 고정되어 있으므로)
    # 따라서, 루트n까지의 검색 범위만 가지면 된다.	

    ## 시간 복잡도가 높은 부분. 최소한으로 search 범위를 좁혀야 한다.
    # 모든 짝수는 제외: range(3, x, 2) - O(N/2)로 여전히 O(N)
    # nat_num의 제곱근까지만 search 하면 된다 : O(sqrt(N))
    reduc_num = round(nat_num ** 0.5)
    for i in range(3, reduc_num+1, 2):
        if nat_num % i is 0:
            return False   
    return True

In [27]:
print(check_prime(8))
print(check_prime(19))
print(check_prime((2 ** 31)-1))

False
True
True


## 2. (조건) 우측 끝에서 임의의 개수의 숫자를 제거한 결과도 소수인 수를 구하시오.

예) 자연수 5939333 는 593933, 59393, 5939, 593, 59, 5 모두 소수이다.

### 2.1. 입력된 자연수보다 적으면서 위의 조건을 만족하는 가장 큰 자연수를 구하시오.

In [20]:
def find_prime_condition(org_input):
    input_ = org_input
    
    while input_: # 입력이 감소하여 0이 되면 break
        str_input = str(input_) 
        tkn = False # sub_set 모두 소수일 경우 true. (ex. [2, 29, 293, 2939, 29399])
        
        # 맨 앞자리가 1 또는 2이면 무조건 소수가 된다.
        #if str_input[0] == '1' or str_input[0] == '2': # 이 연산이 오히려 시간 복잡도를 매우 늘림.
        #    input_ -= 1
        #    continue
        
        sub_set = [int(str_input[:i+1]) for i, _ in enumerate(str_input)] # 현재 입력에 대한 sub_set을 생성.
        #print(sub_set)
        
        for i, each in enumerate(sub_set): # sub_set의 작은 수부터 iteration되도록.(큰 단위로 감소시키기 위해 - search space가 매우 줄어듦.)
            if check_prime(each)==False:
                each -= 1 # 현재 레벨이 소수가 아니므로, 현재 레벨에서 1을 감소시킴
                input_ = int(str(each) + '9'*len(str(str_input[i+1:])) ) # 1을 감소시킨 값을 입력에 반영. 주의: 뒷부분은 모두 9로 이어줘야 함.
                break
                
            if i==len(sub_set)-1: # sub_set의 마지막 elemt까지 소수가 true일 경우.
                tkn = True
        
        # inner while을 모두 통과하면 모든 조건을 만족하는 소수이므로 해당 값을 리텀함.
        if tkn == True:
            return input_
        
def main(nat_num):
    print(find_prime_condition(nat_num))

In [26]:
start_t = time.time()
main((2 ** 31)-1)
main(7000000)
main(46764)
print(" %s seconds " % (time.time() - start_t))

73939133
5939333
37397
 0.031145095825195312 seconds 


### 2.2. 입력된 자연수보다 작은 수 중에서 위의 조건을 만족하는 자연수를 모두 구하시오.

In [28]:
def find_prime_condition2(org_input):
    input_ = org_input
    all_prime = []
    
    while input_: # 입력이 감소하여 0이 되면 break
        
        str_input = str(input_)
        tkn = False # sub_set 모두 소수일 경우 true. (ex. [2, 29, 293, 2939, 29399])
        
        # 맨 앞자리가 1 또는 2이면 무조건 소수가 된다.
        #if str_input[0] == '1' or str_input[0] == '2':
        #    input_ -= 1
        #    continue
        
        sub_set = [int(str_input[:i+1]) for i, _ in enumerate(str_input)]
        
        #print(sub_set)
        
        for i, each in enumerate(sub_set):
            if check_prime(each)==False:
                each -= 1
                input_ = int(str(each) + '9'*len(str(str_input[i+1:])) )
                break
                
            if i==len(sub_set)-1:
                tkn = True
        
        # inner while을 모두 통과하면 모든 조건을 만족하는 소수이므로 해당 값을 리텀함.
        if tkn == True:
            all_prime.append(input_)
            input_ -= 1 ### problem3_2를 위한 추가.
            
        #input_ -= 1
    return all_prime

def main(nat_num):
    print(find_prime_condition2(nat_num))

In [29]:
main((2 ** 31)-1)

[73939133, 59393339, 37337999, 29399999, 23399339, 7393933, 7393931, 7393913, 5939333, 3733799, 2939999, 2399333, 2339933, 739399, 739397, 739393, 739391, 719333, 593993, 593933, 373393, 373379, 293999, 239933, 233993, 73939, 73331, 71933, 59399, 59393, 37397, 37339, 37337, 31379, 31193, 29399, 23993, 23399, 23339, 23333, 7393, 7333, 7331, 7193, 5939, 3797, 3793, 3739, 3733, 3137, 3119, 2939, 2399, 2393, 2339, 2333, 797, 739, 733, 719, 599, 593, 379, 373, 317, 313, 311, 293, 239, 233, 79, 73, 71, 59, 53, 37, 31, 29, 23, 7, 5, 3, 2]
